<a href="https://colab.research.google.com/github/NachoC20/RiesgoMercado/blob/main/VaR_CVaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgoMercado
!git clone "https://github.com/NachoC20/RiesgoMercado"

Cloning into 'RiesgoMercado'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


# Tarea 1: Riesgo de Mercado

In [ ]:
%%R
#Instalamos los paquetes que necesitaremos.
install.packages("quantmod")
install.packages("data.table")
install.packages("PerformanceAnalytics")
install.packages("Deriv")

#Cargamos librerias
library(Deriv)
library(quantmod)
library(data.table)
library("PerformanceAnalytics")
library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [ ]:

%%R
## GENERAMOS LAS FUNCIONES QUE NECESITAREMOS:

# Función de interpolación de tasas por el método alamabrada
talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Función para diagonalizar un vector
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

## **Definición de parámetros para valorar**

In [ ]:
%%R

fval=as.Date("20211015",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo

# Inciso 1, carga de datos
1000 acciones de GCarso, -5000 de AméricaMóvil y 1200 WalMart. *Todas de la BMV*

In [ ]:
%%R
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#Claves de cotización de las emisoras, en orden alfabético
pos_eq=c(-5000,1000,1200) #Monto inicial invertido en acciones
nh=3660 #Días de historia

# Ejercicio 3
2 bonos largos, uno cupón cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 días, un bono M con tasa fija de 4.5% con un cupon de 4.05% anual vencimiento de 3600 días y un bono corto 1000 bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente). 

In [ ]:
%%R

#CETES
base="RiesgosFinancieros/2020-2/Insumos/tasa_guber.txt"
nocional 

#BONDES D
btasadescst="RiesgosFinancieros/2020-2/Insumos/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-2/Insumos/tfondeo.txt"
plazos_bdm=cbind( 358,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon_bdm=cbind( 28,	28, 28, 28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(220, -290, 290, -460, 100) #posición invertida
nominal_bdm=100